In [1]:
from C_Classes import SUBMISSION_510K, SUBMISSION_DE_NOVO

/Users/adam/Projects/23.10.26_FDA_Scraper/.env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# Number = 'K232699' #Example of a 510(k) with a machine-readible PDF
# Number = 'K991766' #Example of a 510(k) with a photo-copied and not machine-readible PDF
# Number = 'DEN130045' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN130045
# Number = 'DEN180044' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN180044
# Number = 'DEN200030' # https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/denovo.cfm?id=DEN200030

# 510k class
# sub1 = SUBMISSION_510K(Number)
# sub1.get_510k_summary()
# sub1.get_510k_pdf()
# sub1.get_510k_IFU()
# sub1.find_potential_predicates(sub1.PDF_Location)
# sub1.summarize()

# #De Novo class
# sub2 = SUBMISSION_DE_NOVO(Number)
# sub2.Submission_Type
# sub2.Submission_Number
# sub2.get_De_Novo_summary()
# sub2.get_Reclassification_pdf()
# sub2.get_FDA_Review_pdf()
# sub2.get_De_Novo_IFU()
# sub2.find_potential_predicates(sub2.PDF_Review_Location)
# sub2.summarize()


This does not look like a 510k Submission number!


In [ ]:
# https://www.fda.gov/about-fda/cdrh-transparency/evaluation-automatic-class-iii-designation-de-novo-summaries
# https://www.fda.gov/medical-devices/510k-clearances/downloadable-510k-files


In [6]:
import numpy as np
import pandas as pd
import requests, zipfile, io

In [ ]:
def download_FDA_file(file):
    URL = 'https://www.accessdata.fda.gov/premarket/ftparea/' + file + '.zip'
    file = './FDA_database_files/' + file + '.txt' 
    r = requests.get(URL)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("./FDA_database_files/")
    return file

def create_FDA_dataframe(file):
    return pd.read_csv(file, sep='|', index_col=False, encoding='cp1252')
    

In [35]:
# Download FDA Cleared 510(k) Submissions
FDA_files = ['pmnlstmn','pmn96cur','pmn9195','pmn8690','pmn8185','pmn7680']
FDA_local_files = [download_FDA_file(file) for file in FDA_files]
FDA_dataframes = [create_FDA_dataframe(file) for file in FDA_local_files]
df = pd.concat(FDA_dataframes)


In [44]:
df.shape

(167285, 22)

Tracing Predicate Devices

In [49]:
import re

In [58]:
def initialize_class_for_submission(submission_number):
    if  re.match('[K,k]\d{6}', submission_number): 
        sub = SUBMISSION_510K(submission_number)
        sub.get_510k_summary()
        sub.get_510k_pdf()
        # sub.get_510k_IFU()
        sub.find_potential_predicates(sub.PDF_Location)
    elif re.match('[D,d][E,e][N,n]\d{6}', submission_number): 
        # #De Novo class
        sub = SUBMISSION_DE_NOVO(submission_number)
        sub.get_De_Novo_summary()
        sub.get_Reclassification_pdf()
        sub.get_FDA_Review_pdf()
        # sub.get_De_Novo_IFU()
        sub.find_potential_predicates(sub.PDF_Review_Location)
        sub.summarize()
    else:
        print("This does not look like a 510k Submission number!")
        sub = None

    return sub

In [59]:
starting_submission = 'K232699'
starting_sub = initialize_class_for_submission(starting_submission)

In [60]:
first_layer = []
list_of_edges = []
for predicate in starting_sub.Potential_Predicates:
    print(predicate)
    predicate_sub = initialize_class_for_submission(predicate)
    list_of_edges.append(starting_sub.Submission_Number + ' -> ' + predicate_sub.Submission_Number)
    first_layer.append(predicate_sub)

    

K210392
K201446
DEN230003
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/adam/Projects/23.10.26_FDA_Scraper/.env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/s6/qclkpdmx04q8gw41wnk6rb1h0000gn/T/ipykernel_60499/827293568.py", line 5, in <module>
    predicate_sub = initialize_class_for_submission(predicate)
  File "/var/folders/s6/qclkpdmx04q8gw41wnk6rb1h0000gn/T/ipykernel_60499/1692115798.py", line 13, in initialize_class_for_submission
    sub.get_FDA_Review_pdf()
  File "/Users/adam/Projects/23.10.26_FDA_Scraper/C_Classes.py", line 183, in get_FDA_Review_pdf
AttributeError: 'SUBMISSION_DE_NOVO' object has no attribute 'FDA_Review_URL'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/adam/Projects/23.10.26_FDA_Scraper/.env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = s

In [57]:
list_of_edges

['K232699 -> K210392', 'K232699 -> K201446']

In [62]:
from C_Classes import SUBMISSION

import re
import sys
import requests
from PyPDF2 import PdfReader 
from bs4 import BeautifulSoup

from F_Functions import find_substring_in_string, clean_IFU, clean_end, clean_start


In [ ]:

class SUBMISSION_510K(SUBMISSION):
    def __init__(self, number):
        SUBMISSION.__init__(self, number)
        self.Submission_Type = "510K"
        if not re.match('[K,k]\d{6}', number): print("This does not look like a 510k Submission number!")

    def get_510k_summary(self):
        self.URL = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpmn/pmn.cfm?ID=' + self.Submission_Number
        page = requests.get(self.URL)

        soup = BeautifulSoup(page.content, "html.parser")
        tables = soup.find_all('table')
        #looks like the 7th table is the one I want
        rows = tables[7].find_all('tr')

        for row in rows[1:]:
            try:
                header = row.find("th")
                data = row.find('td')
                if header.text == "510(k) Number": pass
                else: self.__setattr__(header.text.replace(" ","_"),data.text.strip().replace("\n","").replace("\r","").replace('\xa0',"").replace('\t',""))
                if header.text == 'Summary': 
                    link = row.find("a")
                    self.Summary_URL = link.get("href")
            except Exception as e:
                # print(e)
                continue                
    
    def get_510k_pdf(self,output_folder='./output_folder/',silent=True):
        success = self.get_pdf("510k_Summary",self.Summary_URL,output_folder=output_folder,silent=silent)

        if success:
            self.PDF_Downloaded = "Yes"
            self.PDF_Readable = "Unknown"
            self.PDF_Location = output_folder+'510k_Summary_'+str(self.Submission_Number)+".pdf"
        else:
            self.PDF_Downloaded = "No"
            self.PDF_Readable = "Unknown"
            self.PDF_Location = "NA"

    def get_510k_IFU(self):
        # creating a pdf reader object 
        reader = PdfReader(self.PDF_Location) 
                        
        # extracting text from page 
        page = reader.pages[1]
        text = page.extract_text() 

        if text == "": 
            self.PDF_Readable = "No"
            self.IFU = "Cannot Read PDF."

        else:
            self.PDF_Readable = "Yes"

            all_text = ""
            for page in reader.pages:
                text = page.extract_text()
                all_text = all_text + text

            found, found_snippets = find_substring_in_string("Indications for Use", all_text)
            # print(found_snippets)
            if found:
                for snippet in found_snippets:
                    found, IFU = find_substring_in_string("Type of Use", snippet)
                    if found: 
                        temp1 = clean_IFU(IFU[0],'Indications for Use',"")
                        temp2 = clean_end(temp1)
                        temp3 = clean_start(temp2)
                        self.IFU = temp3
            else: 
                self.IFU = "Not Automatically Found."
    


In [66]:
class SUBMISSION_PMA(SUBMISSION):
    def __init__(self, number):
        SUBMISSION.__init__(self, number)
        self.Submission_Type = "PMA"
        if not re.match('[P,p]\d{6}', number): print("This does not look like a PMA Submission number!")

    def get_PMA_summary(self):
        self.URL = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpma/pma.cfm?id=' + self.Submission_Number
        page = requests.get(self.URL)

        soup = BeautifulSoup(page.content, "html.parser")
        tables = soup.find_all('table')
        #looks like the 7th table is the one I want
        rows = tables[7].find_all('tr')

        for row in rows[1:]:
            try:
                header = row.find("th")
                data = row.find('td')
                print('-'*30)
                print(header)
                print(data)

                if header.text == "510(k) Number": pass
                else: self.__setattr__(header.text.replace(" ","_"),data.text.strip().replace("\n","").replace("\r","").replace('\xa0',"").replace('\t',""))
                if header.text == 'Summary': 
                    link = row.find("a")
                    self.Summary_URL = link.get("href")
            except Exception as e:
                # print(e)
                continue                


In [68]:
Number = 'P220014'
# Number = 'P220004'
# Number = 'P230002'
# Number = 'P220005'

sub = SUBMISSION_PMA(Number)
sub.get_PMA_summary()

------------------------------
None
<td colspan="2"> </td>
------------------------------
<th align="Left">Device</th>
<td align="Left">CraniSeal Dural Sealant</td>
------------------------------
<th align="Left">Generic Name</th>
<td align="Left">Sealant, dural</td>
------------------------------
<th align="Left">Applicant</th>
<td align="Left">Pramand, LLC<br/>201 Burlington Road, Suite 210<br/>Bedford, MA 01730</td>
------------------------------
<th align="Left">PMA Number</th>
<td align="Left">P220014</td>
------------------------------
<th align="Left">Date Received</th>
<td align="Left">09/02/2022</td>
------------------------------
<th align="Left">Decision Date</th>
<td align="Left">07/06/2023</td>
------------------------------
<th align="Left">Product Code</th>
<td align="Left"><a href="/scripts/cdrh/cfdocs/cfPCD/classification.cfm?start_search=1&amp;ProductCode=NQR" style="text-decoration:underline;" title="Products Classification Database information about NQR">NQR</a> </t

In [69]:
sub.summarize()

Submission_Number: P220014 
Submission_Type: PMA 
URL: https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpma/pma.cfm?id=P220014 
Device: CraniSeal Dural Sealant 
Generic_Name: Sealant, dural 
Applicant: Pramand, LLC201 Burlington Road, Suite 210Bedford,MA01730 
PMA_Number: P220014 
Date_Received: 09/02/2022 
Decision_Date: 07/06/2023 
Product_Code: NQR 
Docket_Number: 23M-3158 
Notice_Date: 07/28/2023 
Advisory_Committee: Neurology 
Expedited_Review_Granted?: No 
Combination_Product: No 
Approval_Order: Approval Order 
Summary: Summary of Safety and Effectiveness 
Summary_URL: https://www.accessdata.fda.gov/cdrh_docs/pdf22/P220014B.pdf 
Labeling: Labeling 
Post-Approval_Study: Show Report Schedule and Study Progress 
Supplements::  

